In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import getpass

username = getpass.getuser()

In [19]:
spark = SparkSession.\
        builder.\
        config('spark.ui.port','0').\
        config('spark.sql.warehouse.dir',f'/user/{username}/warehouse').\
        config('spark.shuffle.useOldFetchProtocol','true').\
        enableHiveSupport().\
        master('yarn').\
        getOrCreate()


## Associating points to the grades in order to calculate the Loan Score

In [20]:
spark.conf.set("spark.sql.unacceptable_rated_pts",0)
spark.conf.set("spark.sql.very_bad_rated_pts",100)
spark.conf.set("spark.sql.bad_rated_pts",250)
spark.conf.set("spark.sql.good_rated_pts",500)
spark.conf.set("spark.sql.very_good_rated_pts",650)
spark.conf.set("spark.sql.excellent_rated_pts",800)

In [21]:
spark.conf.set("spark.sql.unacceptable_grade_pts",750)
spark.conf.set("spark.sql.very_bad_grade_pts",1000)
spark.conf.set("spark.sql.bad_grade_pts",1500)
spark.conf.set("spark.sql.good_grade_pts",2000)
spark.conf.set("spark.sql.very_good_grade_pts",2500)

In [22]:
bad_customer_data_final_df = spark.read.csv("/user/itv015278/lendingclubproject/bad_data/customer_data_final" , header = True , inferSchema = True )

In [23]:
bad_customer_data_final_df.createOrReplaceTempView("bad_data_customer")

In [24]:
spark.sql("USE itv015278_lending_club")

""


## Loan Score Calculation Criteria 1: Payment History(ph)

In [25]:
pending_history_df = spark.sql("""
select c.member_id , case when p.last_payment_amount < (c.monthly_installment * 0.5) then ${spark.sql.very_bad_rated_pts}
                          when p.last_payment_amount >= (c.monthly_installment * 0.5) and p.last_payment_amount < c.monthly_installment then ${spark.sql.bad_rated_pts}
                          when p.last_payment_amount = c.monthly_installment then ${spark.sql.good_rated_pts}
                          when p.last_payment_amount > (c.monthly_installment) and p.last_payment_amount < (c.monthly_installment * 1.5) then ${spark.sql.very_good_rated_pts}
                          when p.last_payment_amount > (c.monthly_installment * 1.50) then ${spark.sql.excellent_rated_pts}
                          else ${spark.sql.unacceptable_rated_pts}
                          end as last_payment_points ,
                      case when p.total_payment_received >= (c.funded_amount * 0.50) then ${spark.sql.very_good_rated_pts}
                           when p.total_payment_received < (c.funded_amount * 0.50) and p.total_payment_received > 0 then ${spark.sql.good_rated_pts}
                           when p.total_payment_received = 0 or p.total_payment_received is NULL then ${spark.sql.unacceptable_rated_pts}
                            end as total_payment_points
                          from loans_repayments p
                          inner join loans c on c.loan_id = p.loan_id
                          left join bad_data_customer bdc on bdc.member_id = c.member_id
                          where bdc.member_id is NULL
""")

In [26]:
pending_history_df.createOrReplaceTempView("ph_pts")

In [29]:
loan_default_history = spark.sql("""
select p.* ,
CASE WHEN d.delinq_2yrs = 0 THEN ${spark.sql.excellent_rated_pts}
     when d.delinq_2yrs BETWEEN 1 and 2 then ${spark.sql.bad_rated_pts}
     when d.delinq_2yrs between 3 and 5 then ${spark.sql.very_bad_rated_pts}
     when d.delinq_2yrs > 5 OR d.delinq_2yrs is NULL then ${spark.sql.unacceptable_rated_pts}
     END as delinq_points ,
CASE WHEN l.public_record = 0 THEN ${spark.sql.excellent_rated_pts}
     when l.public_record BETWEEN 1 and 2 then ${spark.sql.bad_rated_pts}
     when l.public_record between 3 and 5 then ${spark.sql.very_bad_rated_pts}
     when l.public_record > 5 OR l.public_record is NULL then ${spark.sql.unacceptable_rated_pts}
     END as public_record_points ,
CASE WHEN l.public_record_bankruptcies = 0 THEN ${spark.sql.excellent_rated_pts}
     when l.public_record_bankruptcies BETWEEN 1 and 2 then ${spark.sql.bad_rated_pts}
     when l.public_record_bankruptcies between 3 and 5 then ${spark.sql.very_bad_rated_pts}
     when l.public_record_bankruptcies > 5 OR l.public_record_bankruptcies is NULL then ${spark.sql.unacceptable_rated_pts}
     END as public_record_bankruptcies_pts ,
CASE WHEN l.inquiries_last_6months = 0 THEN ${spark.sql.excellent_rated_pts}
     when l.inquiries_last_6months BETWEEN 1 and 2 then ${spark.sql.bad_rated_pts}
     when l.inquiries_last_6months BETWEEN 1 and 2 then ${spark.sql.bad_rated_pts}
     when l.inquiries_last_6months between 3 and 5 then ${spark.sql.very_bad_rated_pts}
     when l.inquiries_last_6months > 5 OR l.inquiries_last_6months is NULL then ${spark.sql.unacceptable_rated_pts}
     END as enquiry_points 
from loans_defaulters_detail_rec_enq_new l
inner join loans_defaulters_delinq_new d on l.member_id = d.member_id
inner join ph_pts p on p.member_id = l.member_id
""")

In [31]:
loan_default_history.createOrReplaceTempView("ldh_ph_pts")

In [34]:
fh_ldh_ph_df = spark.sql("select ldef.*, \
   CASE \
   WHEN LOWER(l.loan_status) LIKE '%fully paid%' THEN ${spark.sql.excellent_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%current%' THEN ${spark.sql.good_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%in grace period%' THEN ${spark.sql.bad_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%late (16-30 days)%' OR LOWER(l.loan_status) LIKE '%late (31-120 days)%' THEN ${spark.sql.very_bad_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%charged off%' THEN ${spark.sql.unacceptable_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   END AS loan_status_pts, \
   CASE \
   WHEN LOWER(a.home_ownership) LIKE '%own' THEN ${spark.sql.excellent_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%rent' THEN ${spark.sql.good_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%mortgage' THEN ${spark.sql.bad_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%any' OR LOWER(a.home_ownership) IS NULL THEN ${spark.sql.very_bad_rated_pts} \
   END AS home_pts, \
   CASE \
   WHEN l.funded_amount <= (a.total_high_credit_limit * 0.10) THEN ${spark.sql.excellent_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.10) AND l.funded_amount <= (a.total_high_credit_limit * 0.20) THEN ${spark.sql.very_good_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.20) AND l.funded_amount <= (a.total_high_credit_limit * 0.30) THEN ${spark.sql.good_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.30) AND l.funded_amount <= (a.total_high_credit_limit * 0.50) THEN ${spark.sql.bad_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.50) AND l.funded_amount <= (a.total_high_credit_limit * 0.70) THEN ${spark.sql.very_bad_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.70) THEN ${spark.sql.unacceptable_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   END AS credit_limit_pts, \
   CASE \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A1' THEN ${spark.sql.excellent_rated_pts} \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A2' THEN (${spark.sql.excellent_rated_pts} * 0.95) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A3' THEN (${spark.sql.excellent_rated_pts} * 0.90) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A4' THEN (${spark.sql.excellent_rated_pts} * 0.85) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A5' THEN (${spark.sql.excellent_rated_pts} * 0.80) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B1' THEN (${spark.sql.very_good_rated_pts}) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B2' THEN (${spark.sql.very_good_rated_pts} * 0.95) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B3' THEN (${spark.sql.very_good_rated_pts} * 0.90) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B4' THEN (${spark.sql.very_good_rated_pts} * 0.85) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B5' THEN (${spark.sql.very_good_rated_pts} * 0.80) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C1' THEN (${spark.sql.good_rated_pts}) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C2' THEN (${spark.sql.good_rated_pts} * 0.95) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C3' THEN (${spark.sql.good_rated_pts} * 0.90) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C4' THEN (${spark.sql.good_rated_pts} * 0.85) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C5' THEN (${spark.sql.good_rated_pts} * 0.80) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D1' THEN (${spark.sql.bad_rated_pts}) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D2' THEN (${spark.sql.bad_rated_pts} * 0.95) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D3' THEN (${spark.sql.bad_rated_pts} * 0.90) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D4' THEN (${spark.sql.bad_rated_pts} * 0.85) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D5' THEN (${spark.sql.bad_rated_pts} * 0.80) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E1' THEN (${spark.sql.very_bad_rated_pts}) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E2' THEN (${spark.sql.very_bad_rated_pts} * 0.95) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E3' THEN (${spark.sql.very_bad_rated_pts} * 0.90) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E4' THEN (${spark.sql.very_bad_rated_pts} * 0.85) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E5' THEN (${spark.sql.very_bad_rated_pts} * 0.80) \
   WHEN (a.grade) in ('F', 'G') THEN (${spark.sql.unacceptable_rated_pts}) \
   END AS grade_pts \
   FROM ldh_ph_pts ldef \
   INNER JOIN loans l ON ldef.member_id = l.member_id \
   INNER JOIN customers_new a ON a.member_id = ldef.member_id  ")

## Loan Score Calculation Criteria 2: Loan Defaulters History(ldh)

In [38]:
fh_ldh_ph_df.createOrReplaceTempView("fh_ldh_ph_pts")

In [45]:
loan_score = spark.sql("SELECT member_id, \
((last_payment_points+total_payment_points)*0.20) as payment_history_points, \
((delinq_points + public_record_points + public_record_bankruptcies_pts + enquiry_points) * 0.45) as defaulters_history_points, \
((loan_status_pts + home_pts + credit_limit_pts + grade_pts)*0.35) as financial_health_points \
FROM fh_ldh_ph_pts")

In [46]:
final_loan_score = loan_score.withColumn('loan_score', loan_score.payment_history_points + loan_score.defaulters_history_points + loan_score.financial_health_points)

In [47]:
final_loan_score.createOrReplaceTempView("loan_score_eval")

In [48]:
loan_score_final = spark.sql("select ls.*, \
case \
WHEN loan_score > ${spark.sql.very_good_grade_pts} THEN 'A' \
WHEN loan_score <= ${spark.sql.very_good_grade_pts} AND loan_score > ${spark.sql.good_grade_pts} THEN 'B' \
WHEN loan_score <= ${spark.sql.good_grade_pts} AND loan_score > ${spark.sql.bad_grade_pts} THEN 'C' \
WHEN loan_score <= ${spark.sql.bad_grade_pts} AND loan_score  > ${spark.sql.very_bad_grade_pts} THEN 'D' \
WHEN loan_score <= ${spark.sql.very_bad_grade_pts} AND loan_score > ${spark.sql.unacceptable_grade_pts} THEN 'E'  \
WHEN loan_score <= ${spark.sql.unacceptable_grade_pts} THEN 'F' \
end as loan_final_grade \
from loan_score_eval ls")

In [51]:
loan_score_final.rdd.getNumPartitions()

200

In [52]:
loan_score_final.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv015278/lendingclubproject/processed/loan_score") \
.save()